In [1]:
import numpy as np 

In [2]:
DNA_SIZE = 24
POP_SIZE = 200
CROSSOVER_RATE = 0.8
MUTATION_RATE = 0.005
N_GENERATIONS = 50
X_BOUND = [-3, 3]
Y_BOUND = [-3, 3]

In [6]:
def F(x,y):
    return 3 * (1 - x)**2 * np.exp(-(x**2) - (y + 1)**2)

def translateDNA(pop):
    x_pop = pop[:,1::2]
    y_pop = pop[:,::2]
    x = x_pop.dot(2**np.arange(DNA_SIZE)[::-1])/ float(2**DNA_SIZE - 1) * (X_BOUND[1] - X_BOUND[0]) + X_BOUND[0]
    y = y_pop.dot(2**np.arange(DNA_SIZE)[::-1])/ float(2**DNA_SIZE - 1) * (Y_BOUND[1] - Y_BOUND[0]) + Y_BOUND[0]
    return x,y

def get_best_individual(pop):
    fitnesses = get_fitness(pop)
    best_idx = np.argmax(fitnesses)
    x,y = translateDNA(pop)
    return x[best_idx],y[best_idx]

def get_fitness(pop):
    x,y = translateDNA(pop)
    pred = F(x,y)
    return (pred - np.min(pred)) + 1e-3
 
def select(pop, fitness):
    idx = np.random.choice(np.arange(POP_SIZE), size=POP_SIZE, replace=True, p=fitness/fitness.sum())
    return pop[idx]

def crossover_and_mutation(pop, CROSSOVER_RAGE = 0.8):
    new_pop = []
    for father in pop:
        child = father
        if np.random.rand() < CROSSOVER_RAGE:
            mother = pop[np.random.randint(POP_SIZE)]
            cross_points = np.random.randint(low=0, high=DNA_SIZE*2)
            child[cross_points:] = mother[cross_points:]
        mutation(child)
        new_pop.append(child)
    return new_pop

def mutation(child, MUTATION_RATE = 0.003):
    if np.random.rand() < MUTATION_RATE:
        mutate_point = np.random.randint(0,DNA_SIZE)
        child[mutate_point] = child[mutate_point] ^ 1


def print_info(pop):
    fitnees = get_fitness(pop)
    max_fitness_index = np.argmax(fitnees)
    x,y = translateDNA(pop)
    print("最优秀的基因:",pop[max_fitness_index])
    print("(x,y):",x[max_fitness_index], y[max_fitness_index])
    print("适应度:",fitnees[max_fitness_index])



In [12]:
def main():
    pop = np.random.randint(2, size=(POP_SIZE, DNA_SIZE*2))
    for i in range(N_GENERATIONS):
        pop = np.array(crossover_and_mutation(pop,CROSSOVER_RATE))
        fitness = get_fitness(pop)
        pop = select(pop,fitness=fitness)
        x,y = get_best_individual(pop)
        best_val = F(x,y)
        #print(f"Generation {i+1}: Best Value = {best_val},x = {x},y = {y}")

    print_info(pop)

main()

最优秀的基因: [0 0 1 1 0 1 1 0 0 0 1 0 0 1 1 1 1 1 1 1 0 1 0 1 0 1 1 0 1 1 0 0 1 1 1 1 0
 1 0 0 1 1 1 1 0 1 0 1]
(x,y): -0.6567137036748947 -0.9896119826800813
适应度: 0.0021275952093638964


In [4]:
import random
import numpy as np

class GeneticAlgorithm:
    # 遗传算法
    POP_SIZE = 100  # 种群数量
    GEN_SIZE = 32  # 基因长度
    CROSS_RATE = 0.8  # 交叉概率
    MUTATE_RATE = 0.01  # 变异概率
    N_ITER = 100  # 迭代次数
    BOUND = [-10, 10]  # 变量取值范围

    def __init__(self, pop_size=POP_SIZE, gen_size=GEN_SIZE, cross_rate=CROSS_RATE, mutate_rate=MUTATE_RATE, n_iter=N_ITER, bound=BOUND):
        self.pop_size = pop_size
        self.gen_size = gen_size
        self.cross_rate = cross_rate
        self.mutate_rate = mutate_rate
        self.n_iter = n_iter
        self.bound = bound
        self.population = self.init_population()



    # 定义目标函数
    def get_fitness(self,x):
        return -x ** 2 + 16 * x - 10

    def translate_pos(self,pop):
        value = pop.dot(2**np.arange(self.gen_size)[::-1]) / float(2**self.gen_size-1) * (self.bound[1] - self.bound[0]) + self.bound[0]
        return value

    def fitness_pop(self,pop):
        value = self.get_fitness(self.translate_pos(pop))
        return value
    
    def init_population(self):
        pop = np.random.randint(2, size=(self.pop_size, self.gen_size))
        return pop

    # 选择函数，轮盘赌选择
    def  select(self,pops):
        fitness = self.fitness_pop(pops)
        fitness = fitness - fitness.min() + 1e-6  # 防止出现负值
        fitness = fitness / fitness.sum()
        idx = np.random.choice(np.arange(self.pop_size), size=self.pop_size, replace=True, p=fitness)
        return pops[idx]


    # 交叉函数，单点交叉
    def crossover(self,parent1, parent2):
        cross_points = np.random.randint(low=0, high=self.gen_size)
        child = parent1.copy()
        child[cross_points:] = parent2[cross_points:]
        return child

    # 变异函数，随机变异
    def mutate(self,individual):
        if np.random.rand() < self.mutate_rate:
            mutate_point = np.random.randint(0,self.gen_size)
            individual[mutate_point] = individual[mutate_point] ^ 1

        return individual

    # 进化函数，迭代进化
    def evolve(self,population):
        new_population = []
        for farther in population:
            child = farther
            if np.random.rand() < self.cross_rate:
                mother = population[np.random.randint(self.pop_size)]
                child = self.crossover(farther, mother)
            self.mutate(child)
            new_population.append(child)

        new_population = self.select(np.array(new_population))

        return new_population

    # 主函数
    def run(self):        
        for i in range(self.n_iter):
            self.population = self.evolve(self.population)
            fitness = self.fitness_pop(self.population)
            best_idx = np.argmax(fitness)
            best_pos = self.translate_pos(self.population[best_idx])
            print(f"第{i+1}代最优解：{best_pos}, 目标函数值：{fitness[best_idx]}")

In [5]:
gen = GeneticAlgorithm()
gen.run()

第1代最优解：-0.17750112111156469, 目标函数值：-12.871524585780897
第2代最优解：-0.11756436203549647, 目标函数值：-11.894851171788757
第3代最优解：-0.11756436203549647, 目标函数值：-11.894851171788757
第4代最优解：-0.11756436203549647, 目标函数值：-11.894851171788757
第5代最优解：-0.05848763279115943, 目标函数值：-10.939222927848064
第6代最优解：-0.056047274278487436, 目标函数值：-10.899897685409847
第7代最优解：-0.056047274278487436, 目标函数值：-10.899897685409847
第8代最优解：-0.056047274278487436, 目标函数值：-10.899897685409847
第9代最优解：-0.05604283652641939, 目标函数值：-10.899826183948637
第10代最优解：-0.06443353138501706, 目标函数值：-11.035088182127017
第11代最优解：-0.06349903765681653, 目标函数值：-11.020016730292406
第12代最优解：-0.06347747288259598, 目标函数值：-11.019668955685097
第13代最优解：-0.2011535899250667, 目标函数值：-13.258920205540809
第14代最优解：-0.20111175025839145, 目标函数值：-13.258233940226257
第15代最优解：-0.20033336947679814, 目标函数值：-13.245467370554698
第16代最优解：-0.20032973731875714, 目标函数值：-13.245407800754316
第17代最优解：-0.19922749842499066, 目标函数值：-13.22733157092853
第18代最优解：-0.19728159769374898, 目标函数值：-13.195425591888583
